In this notebook, we will use Python to calculate the ampacity of a cable based on its thermal resistance and the given ambient temperature and current.

The reference paper is [here](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=e46fd00335c248c5ecbeef370e53374166dc16d4)

Inputs

- The ambient temperature (Ta) in degrees Celsius

- The current (I_given) in amperes

- The conductor resistance (R_dc) in microhms per foot at 25 degrees Celsius

- The cable data, including the diameter of the segmental conductor (Dc), the diameter of the insulation (Di), and the diameter of the sheath (Ds) in inches

- The duct bank data, including the number of conductors per cable (nn), the number of cables per duct (N), the inside diameter of the duct (id_duct), the thickness of the duct (t_duct), and the outside diameter of the duct (od_duct)

- The duct bank layout, including the number of rows (row) and columns (col) of the duct banks, the depth of the reference cable below the earth's surface (L) in inches, the vertical distance between the centers of two duct banks (d_vert) in inches, the horizontal distance between the centers of two duct banks (d_hori) in inches, the short side of the duct (x) in inches, the long side of the duct (y) in inches, and the depth to the center of a duct bank (Lb) in inches

- The thermal resistances of the insulation (rho_insulation), insulation jacket (rho_jacket), fiber duct (rho_duct), earth (rho_earth), and concrete (rho_concrete)

- The concrete/earth thermal resistance parameters, including the loss factor (LF), the fictitious diameter at which the effect of the loss factor commences (Dx), and the diameter at the start of the earth portion of the thermal circuit (De)

In [5]:
import numpy as np

# input1: temperature and current info
Ta = 25     # ambient temperture, degree C
I_given = 700   # in amp

# input2: conductor resistance
R_dc = 8.6   # microhms per foot; at 25 degree C

# input3: cable data: inches
Dc = 1.543 # diameter of the segmental conductor
Di = 2.113 # diameter of insulation, outside
Ds = 2.373 # diameter of sheath, outside

# input4: duct bank data
nn = 1     # number of conductors per cable [abc]
N  = 6     # number of cables/duct
id_duct = 5 # inside diameter of the duct/conduit
t_duct  = 0.25 # thickness of the duct 
od_duct = id_duct+2*t_duct # outside diameter of the duct

# input5: duct bank layout
row = 3  # row of the duct banks
col = 2  # column of the duct banks
L = 43.5 # depth of reference cable below earth's surface, inches
d_vert = 9  # vertical distance between centers of two duct bank
d_hori = 9  # horizontal distance between centers of two duct bank
x = 18      # short side of the duct
y = 27      # long side of the duct
Lb= 43.5    # depth to the center of a duct bank, inches

# input6: thermal resistance: degree-cm/W
rho_insulation = 550  # insulation
rho_jacket = 550      # insulation jacket
rho_duct = 480        # fiber duct
rho_earth = 120       # earth
rho_concrete = 85     # concrete

# input 7: concrete/earth thermal resistance
LF = 0.8 # loss factor
Dx = 8.3 # ficitious diameter at which the effect of loss factor commences
De = od_duct # diameter at start of the earth portion of the thermal circuit

Intermidate variables

- Ri: Thermal resistance of insulation (ohm-foot)

- Rsd: Thermal resistance of segmental conductor (ohm-foot)

- Rd: Thermal resistance of fiber duct (ohm-foot)

- F: Mutual heating factor
 
- Gb: Duct bank geometric factor

- Re: Thermal resistance for the whole duct bank including the earth on top (ohm-foot)

- Rca: Total thermal resistance (ohm-foot)

In [6]:
# cal1: calculate the thermal resistance: Ri, Rsd, Rd
Ri = 0.012*rho_insulation*np.log10(Di/Dc) # thermal ohm-foot
Ap = 4.6    # default parameter 1
Bp = 0.27   # default parameter 2
Rsd = nn*Ap/(Ds+Bp) # thermal ohm-foot
De = od_duct
Rd = 0.0104*rho_duct*nn*t_duct/(od_duct-t_duct)
print("Ri: " + str(Ri))
print("Rsd: " + str(Rsd))
print("Rd: " + str(Rd))


# cal2: function in calculating the mutual heating factor F
def cal_mut_heat(row, col, L, d_vert, d_hori): 
    N = row*col     # total # of duct banks
    d_diag = (d_vert**2 + d_hori**2)**0.5
    d_vec = np.array([1, d_vert, d_vert, d_hori, d_diag, d_diag])
    d_diag_short = (d_hori**2 + (2*L-d_vert)**2)**0.5
    d_diag_mid   = (d_hori**2 + (2*L)**2)**0.5
    d_diag_long  = (d_hori**2 + (2*L+d_vert)**2)**0.5
    D_vec = np.array([1, 2*L-d_vert, 2*L+d_vert, d_diag_short, d_diag_mid, d_diag_long])
    product1 = np.prod(d_vec)
    product2 = np.prod(D_vec)
    F = round(product2/product1)

    return F

F = cal_mut_heat(row, col, L, d_vert, d_hori)
print("Mutual heating factor F:" + str(F))

# cal3: function in calculating the geometric factor Gb
def cal_geo_factor(x, y, Lb):
    rb_c1 = x/y/2*(4/np.pi-x/y)
    rb_c2 = np.log10(1+y**2/x**2)
    rb_c3 = np.log10(x/2)   
    rb = 10**(rb_c1*rb_c2+rb_c3)
    u = Lb/rb         # 3.4
    Gb =  np.log10(u+(u**2-1)**0.5) # geometric factor applying to a duct bank 
    Gb = round(Gb, 2)

    return Gb

Gb = cal_geo_factor(x, y, Lb)
print("Duct bank geometric factor Gb: " + str(Gb)) # now match the table result

# cal4: thermal resistance for the whole duct bank including the earth on top
Re_1 = np.log10(Dx/De)
Re_2 = LF*np.log10(4*L/Dx*F)
Re_corr = 0.012*(rho_earth-rho_concrete)*nn*N*LF*Gb
Re = 0.012*rho_concrete*nn*(Re_1+Re_2)+Re_corr
print("Re: " + str(Re))

# the total thermal resistance
Rca = Ri + Rsd + Rd + Re
print("Rca: " + str(Rca))

Ri: 0.9011215685284898
Rsd: 1.740446462353386
Rd: 0.2377142857142857
Mutual heating factor F:41985
Duct bank geometric factor Gb: 0.87
Re: 6.786975006697016
Rca: 9.666257323293177


Final output

- Tc: Cable core temperature (in degree C)

In [7]:
# cal5: calculate the temperature given the current
print("The given current is: " + str(I_given) + "(in amp)")
Tc = Ta + (I_given/1000)**2*R_dc*Rca  # in deg C
Tc = round(Tc, 1)
print("Cable core temperature is: " + str(Tc) + "(in deg C)")

The given current is: 700(in amp)
Cable core temperature is: 65.7(in deg C)
